In [22]:
import simulator
import datetime
import importlib
importlib.reload(simulator)
from component import *
from simulator import Environment

# 创建了一个环境
env = Environment(truck_per_station=100,line_size=2,running_day=2,start_date=datetime.datetime(2024, 5, 1))


# simulator.py的一开始定义了很多参数,可查看一下

# 这个是环境中的厂站、工地和合同关系
stations = env.stations
projects = env.projects
interactions = env.interaction


# 初始化Dispatch列表
dispatch_list :List[Dispatch]= []


# ----------------------整个流程，以第1天为例----------------------


env.reset2()

# 返回第1天的计划订单
plan_orders = env.return_next_day_plan_orders(datetime.datetime(2024, 5, 1),day_0=True)

# 安排发车计划，这里随机安排
for order in plan_orders:
    oid = order.oid
    n_need = order.n_need
    
    pid = order.pid
    
    avalible_stations = list(env.interaction[pid].keys())
    import random
    
    chosen_sid = random.choice(avalible_stations)
    
    for i in range(n_need):
        dispatch_list.append(Dispatch(oid=oid,pid=order.pid,from_sid=chosen_sid,dispatch_time=order.plan_arrive_time,ret_sid=chosen_sid))
        
        
env.running_one_day(datetime.datetime(2024, 5, 1),dispatchs=dispatch_list,instant_orders_dispatch_method=None,reposition_method=None)



# ----------------可重复多天尝试---------------
# ----------------连续尝试多天只要reset2()一次即可----------------


print(env.revenue)
print(env.cost)
print(env.penalty)
print(env.count)


/home/jiangrenchi/RL/simulator.py:1261: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  filtered_df = filtered_df[(filtered_df['deliver_time'].dt.hour >= self.order_range[0]) & (df['deliver_time'].dt.hour < self.order_range[1])]


Dispatchs can satisfy the orders, dispatchs and needs are equal.
1920800.0
66297.5999999998
{'waiting': 0, 'overtime': 0, 'discontinuity': 0}
{'overtime': 4, 'discontinuity': 0}


In [61]:
import torch
import time

# 设置设备
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {DEVICE}")

# 生成模拟数据（三个大型向量，首维为3）
size = 100  # 1000万元素，模拟大数据场景
vector1 = torch.rand(3, size, dtype=torch.float32)
vector2 = torch.rand(3, size, dtype=torch.float32)
vector3 = torch.rand(3, size, dtype=torch.float32)

# 方式1: 先切片再传GPU
torch.cuda.synchronize()
start1 = time.time()

tensors1 = []
for i in range(3):
    # 切片并创建tensor
    tensor1 = torch.tensor(vector1[i:i+1], dtype=torch.float32, device=DEVICE)
    tensor2 = torch.tensor(vector2[i:i+1], dtype=torch.float32, device=DEVICE)
    tensor3 = torch.tensor(vector3[i:i+1], dtype=torch.float32, device=DEVICE)
    tensors1.append((tensor1, tensor2, tensor3))

torch.cuda.synchronize()
time1 = time.time() - start1

# 方式2: 先传GPU再切片
torch.cuda.synchronize()
start2 = time.time()

# 先将整个向量传到GPU
vector1_gpu = vector1.to(DEVICE)
vector2_gpu = vector2.to(DEVICE)
vector3_gpu = vector3.to(DEVICE)

tensors2 = []
for i in range(3):
    # 在GPU上直接切片
    tensor1 = vector1_gpu[i:i+1]
    tensor2 = vector2_gpu[i:i+1]
    tensor3 = vector3_gpu[i:i+1]
    tensors2.append((tensor1, tensor2, tensor3))

torch.cuda.synchronize()
time2 = time.time() - start2

# 计算比值
speedup = time1 / time2

print(f"\n数据规模: 3 x {size} 个浮点数")
print(f"方式1 (先切片再传GPU): {time1:.6f} 秒")
print(f"方式2 (先传GPU再切片): {time2:.6f} 秒")
print(f"方式2比方式1快 {speedup:.2f} 倍")

Using device: cuda:0

数据规模: 3 x 100 个浮点数
方式1 (先切片再传GPU): 0.024760 秒
方式2 (先传GPU再切片): 0.003265 秒
方式2比方式1快 7.58 倍


/tmp/ipykernel_1753107/1362606123.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tensor1 = torch.tensor(vector1[i:i+1], dtype=torch.float32, device=DEVICE)
/tmp/ipykernel_1753107/1362606123.py:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tensor2 = torch.tensor(vector2[i:i+1], dtype=torch.float32, device=DEVICE)
/tmp/ipykernel_1753107/1362606123.py:23: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tensor3 = torch.tensor(vector3[i:i+1], dtype=torch.float32, device=DEVICE)
